In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Dropout
from tensorflow.keras.models import Model 
from tensorflow.keras.applications.inception_v3 import InceptionV3 
from tensorflow.keras.applications.inception_v3 import preprocess_input 
from tensorflow.keras.preprocessing import image 
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img 
from tensorflow.keras import models 
import tensorflow as tf 
import os
import tensorflow_datasets as tfds
import numpy as np
import glob 
import matplotlib.pyplot as plt 
from keras.models import load_model
import ntpath 
from sklearn.metrics import confusion_matrix

In [4]:
%cd /content/drive/MyDrive/AgriWise/ML/

/content/drive/.shortcut-targets-by-id/1nNpHsxhVob0qpN6DIX6h02v09VRuwzHw/AgriWise/ML


In [5]:
im_size = 255
batch_size = 50

In [ ]:
train_set = tf.keras.preprocessing.image_dataset_from_directory(
   "/content/drive/MyDrive/AgriWise/ML/plant diseases/train",
    shuffle = True, 
    image_size = (im_size, im_size),
    batch_size = batch_size
)

Found 70298 files belonging to 38 classes.


In [6]:
test_set = tf.keras.preprocessing.image_dataset_from_directory(
   "/content/drive/MyDrive/AgriWise/ML/plant diseases/test",
    shuffle = True, 
    image_size = (im_size, im_size),
    batch_size = batch_size
)

Found 17572 files belonging to 38 classes.


In [8]:
categories = test_set.class_names
categories

['Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Blueberry___healthy',
 'Cherry_(including_sour)___Powdery_mildew',
 'Cherry_(including_sour)___healthy',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn_(maize)___Common_rust_',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Corn_(maize)___healthy',
 'Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Grape___healthy',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Peach___Bacterial_spot',
 'Peach___healthy',
 'Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Raspberry___healthy',
 'Soybean___healthy',
 'Squash___Powdery_mildew',
 'Strawberry___Leaf_scorch',
 'Strawberry___healthy',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_

In [9]:
five_plantsCategories = ['Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn_(maize)___Common_rust_',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Corn_(maize)___healthy',
 'Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Grape___healthy',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_spider_mite',
 'Tomato___Target_Spot',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 'Tomato___Tomato_mosaic_virus',
 'Tomato___healthy']

In [ ]:
import shutil

def TrainTestFolders(train_set, categories, val_ratio = 0.2):

  root_dir = '/content/drive/MyDrive/AgriWise/ML/plant diseases/'
  train_dir = 'train/'
  trainVal_dir = 'Train Val/'

  # create folders 
  for category in categories:
      # make new directories for train and val outside train folder 
      os.makedirs(root_dir + trainVal_dir + 'train/' + category)
      os.makedirs(root_dir + trainVal_dir + 'val/' + category)

      # category file path and name: source
      source = root_dir + train_dir + category
      print("source: ", source)

      # images names
      fileNames = os.listdir(source)
      print(fileNames)
      np.random.shuffle(fileNames)

      # split images into train and val
      train_FileNames, val_FileNames = np.split(np.array(fileNames), [int(len(fileNames) * (1 - val_ratio))])

      # file names from array to list
      train_fileNames = [source +'/'+ name for name in train_FileNames.tolist()]
      val_fileNames = [source +'/'+ name for name in  val_FileNames.tolist()]

      print('Total images  : '+  category + ' ' +str(len(fileNames)))
      print('Training : '+ category + ' '+ str(len(train_fileNames)))
      print('Validation : '+ category + ' ' +str(len(val_fileNames)))

      # copy images to new folders
      for name in train_fileNames:
          shutil.copy(name , root_dir + trainVal_dir + 'train/' + category)

      for name in val_fileNames:
          shutil.copy(name , root_dir + trainVal_dir + 'val/' + category)

In [ ]:
TrainTestFolders(train_set, five_plantsCategories)

source:  /content/drive/MyDrive/AgriWise/ML/plant diseases/train/Apple___Apple_scab
['9f76829c-655e-4f75-b5fd-2ddfd01f963a___FREC_Scab 3337_new30degFlipLR.JPG', 'a72a9033-d2b9-4c64-93bc-7e5a1a0ea4ae___FREC_Scab 3284_new30degFlipLR.JPG', '80ddd81b-9c75-4711-8012-7ee68af378db___FREC_Scab 3458_270deg.JPG', 'adfdba4a-a725-4c5b-8b1c-96e8b20baa7b___FREC_Scab 3265_90deg.JPG', '80894f41-f887-4d0c-8b09-0a7cd24fbe18___FREC_Scab 3526_new30degFlipLR.JPG', '900e46e0-2f82-4062-8963-50fa06834c38___FREC_Scab 3301.JPG', 'aba69098-9c79-46cc-ba1f-6f88a456b56f___FREC_Scab 3039.JPG', '8a8f8dd2-a938-4288-a8ec-56ef0aaa2d4f___FREC_Scab 2947.JPG', '85aad925-f1e2-463e-840b-266d49a56d1f___FREC_Scab 3100_90deg.JPG', 'a2d74032-fd6c-469b-897a-054705e03dec___FREC_Scab 3028.JPG', '84ef6106-d401-44fb-a5c9-be5ef93cb4e7___FREC_Scab 2921_270deg.JPG', '8632c6b9-84e1-4779-9294-71548ff838e5___FREC_Scab 3423_new30degFlipLR.JPG', '9efcd800-e927-4595-8eb8-26d36524242a___FREC_Scab 3026_90deg.JPG', '95c2cc04-5581-48ca-8001-9644b

In [25]:
import shutil

def TestFolder(test_set, categories):

  root_dir = '/content/drive/MyDrive/AgriWise/ML/plant diseases/'
  test_dir = 'test/'
  fiveClasses = 'fiveClasses/'

  # create folders 
  for category in categories:
      # make new directories for test data outside test folder 
      os.makedirs(root_dir + fiveClasses + 'test/' + category)

      # category file path and name: source
      source = root_dir + test_dir + category
      print("source: ", source)

      # images names
      fileNames = os.listdir(source)

      # file names from array to list
      test_fileNames = [source +'/'+ name for name in fileNames]

      print('Total images  : '+  category + ' ' +str(len(fileNames)))

      # copy images to new folders
      for name in test_fileNames:
          shutil.copy(name , root_dir + fiveClasses + 'test/' + category)

In [26]:
TestFolder(test_set, five_plantsCategories)

source:  /content/drive/MyDrive/AgriWise/ML/plant diseases/test/Apple___Apple_scab
Total images  : Apple___Apple_scab 504
source:  /content/drive/MyDrive/AgriWise/ML/plant diseases/test/Apple___Black_rot
Total images  : Apple___Black_rot 497
source:  /content/drive/MyDrive/AgriWise/ML/plant diseases/test/Apple___Cedar_apple_rust
Total images  : Apple___Cedar_apple_rust 440
source:  /content/drive/MyDrive/AgriWise/ML/plant diseases/test/Apple___healthy
Total images  : Apple___healthy 502
source:  /content/drive/MyDrive/AgriWise/ML/plant diseases/test/Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot
Total images  : Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot 410
source:  /content/drive/MyDrive/AgriWise/ML/plant diseases/test/Corn_(maize)___Common_rust_
Total images  : Corn_(maize)___Common_rust_ 477
source:  /content/drive/MyDrive/AgriWise/ML/plant diseases/test/Corn_(maize)___Northern_Leaf_Blight
Total images  : Corn_(maize)___Northern_Leaf_Blight 477
source:  /content/drive/My